## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-15-48-07 +0000,wapo,Trump threatens BBC with lawsuit amid editing ...,https://www.washingtonpost.com/business/2025/1...
1,2025-11-10-15-47-39 +0000,nyt,A Million Evacuated as Typhoon Fung-Wong Hit t...,https://www.nytimes.com/2025/11/10/world/asia/...
2,2025-11-10-15-47-19 +0000,bbc,Homelessness charity to become landlord for fi...,https://www.bbc.com/news/articles/c0ex5r4q82qo...
3,2025-11-10-15-42-39 +0000,nyt,Syria al-Shara al-Baghdadi Trump,https://www.nytimes.com/2025/11/10/world/middl...
4,2025-11-10-15-40-31 +0000,cbc,U.S. Supreme Court rejects bid to overturn sam...,https://www.cbc.ca/news/world/supreme-court-sa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,41
88,shutdown,29
26,court,13
122,government,13
58,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,102
247,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,101
17,2025-11-10-15-18-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,92
9,2025-11-10-15-32-00 +0000,wsj,The BBC reported that President Trump has thre...,https://www.wsj.com/politics/policy/bbc-chairm...,88
210,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,102,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...
28,59,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
17,50,2025-11-10-15-18-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...
61,48,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
252,40,2025-11-09-18-33-00 +0000,nypost,BBC head Tim Davie resigns over doctored foota...,https://nypost.com/2025/11/09/world-news/bbc-h...
125,35,2025-11-10-09-54-20 +0000,cbc,"Typhoon Fung-wong blows away from Philippines,...",https://www.cbc.ca/news/world/typhoon-fung-won...
201,33,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
247,31,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
83,30,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...
199,28,2025-11-10-00-12-31 +0000,nypost,Boca Raton makes serious pitch to NYC business...,https://nypost.com/2025/11/09/us-news/boca-rat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
